In [1]:
# DELETE BEFORE PUBLISHING
# This is just here so you can preview the styling on your local machine

from IPython.core.display import HTML
HTML("""
<style>
.usecase-title, .usecase-duration, .usecase-section-header {
    padding-left: 15px;
    padding-bottom: 10px;
    padding-top: 10px;
    padding-right: 15px;
    background-color: #0f9295;
    color: #fff;
}

.usecase-title {
    font-size: 1.7em;
    font-weight: bold;
}

.usecase-authors, .usecase-level, .usecase-skill {
    padding-left: 15px;
    padding-bottom: 7px;
    padding-top: 7px;
    background-color: #baeaeb;
    font-size: 1.4em;
    color: #121212;
}

.usecase-level-skill  {
    display: flex;
}

.usecase-level, .usecase-skill {
    width: 50%;
}

.usecase-duration, .usecase-skill {
    text-align: right;
    padding-right: 15px;
    padding-bottom: 8px;
    font-size: 1.4em;
}

.usecase-section-header {
    font-weight: bold;
    font-size: 1.5em;
}

.usecase-subsection-header, .usecase-subsection-blurb {
    font-weight: bold;
    font-size: 1.2em;
    color: #121212;
}

.usecase-subsection-blurb {
    font-size: 1em;
    font-style: italic;
}
</style>
""")

<div class="usecase-title">Explorer template</div>

<div class="usecase-authors"><b>Authored by: </b>Keefe Alpay and Nathan Clee</div>

<div class="usecase-duration"><b>Duration:</b>20 mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b>Level: </b>Intermediate</div>
    <div class="usecase-skill"><b>Pre-requisite Skills: </b>Python, Folium, Pandas</div>
</div>

<div class="usecase-section-header">Scenario</div>

* This is not a use case, this is a template of a use case that when you follow the steps you can make a use case which will show you all of the locations of what you want to see and put into the API and where the closet 5 are and where the best trail is from you. 
* To use this template pick a dataset, or multiple and follow the steps. You will need a google maps API as well.
* For an example of how the use cases can be made, used or modified look at Urban Art Explorer and Bar Explorer.



## This block calls your API

In [ ]:
import requests
import pandas as pd

response = requests.get(f'Your APi here') #Your API here
data = response.json()

ID = []
for record in data['records']:
    name = record['fields'].get('change') #change to whatever the name of your item is. 
    description = record['fields'].get('change') #change to whatever the description of your item is, if none remove. 
    coordinates = record['fields'].get('change') #change to whatever the Coordinates of your item is. 
    
    ID.append({
        'Name': name,
        'Address': description,
        'Coordinates': coordinates,
    })
    
df1 = pd.DataFrame(ID)

## This block calls your second API, If your doing one dataset delete this, if you are doing more then two copy this and change the df(add number here)

In [ ]:
import requests
import pandas as pd

response = requests.get(f'Your APi here') #Your API here
data = response.json()

ID = []
for record in data['records']:
    ID.append({
        'Title': record['fields']['change'], #change to whatever the name of your item is. 
        'Description': record['fields']['change'], #change to whatever the description of your item is, if none remove. 
        'Coordinates': record['fields']['change'], #change to whatever the Coordinates of your item is. 
    })

df2 = pd.DataFrame(ID)

## This block combines dfs and removes duplicates and resets the index. If you only have one DF remove this whole block. 

In [ ]:
import pandas as pd
import math
import requests

df_combined = pd.concat([df1, df2])

df_combined = df_combined.drop_duplicates(subset='Title')
df_combined = df_combined.drop_duplicates(subset='Coordinates')
df_combined = df_combined.drop_duplicates(subset='Description')

Co = df_combined['Coordinates']

df_combined['Coordinates'] = df_combined['Coordinates'].apply(lambda x: (round(x[0], 6), round(x[1], 6)))
df_combined = df_combined.drop_duplicates(subset='Coordinates')

## This block takes a users address and gives table with information and a map of the closets 5 (whatever your looking for) to the user.

## To make this work you need a Google API and to have all the libraries installed. 

In [ ]:
import pandas as pd
import math
import requests
from IPython.display import display
from IPython.display import HTML
import folium

address = input("Please enter the address: ")

api_key = ''  # add your google api here

def get_coordinates_from_address(address, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            lat = data['results'][0]['geometry']['location']['lat']
            lng = data['results'][0]['geometry']['location']['lng']
            return lat, lng
    return None

def compute_distance(coord1, coord2):
    R = 6371  

    lat1, lon1 = coord1
    lat2, lon2 = coord2

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * 
         math.sin(dlon / 2) * math.sin(dlon / 2))
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c

    return distance * 1000 

set_coordinates = get_coordinates_from_address(address, api_key)
df1['Distance'] = df1['Coordinates'].apply(lambda x: compute_distance(x, set_coordinates))
closest = df1.nsmallest(5, 'Distance')

def get_address_from_coordinates(lat, lng, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            return data['results'][0]['formatted_address']
    return None

mymap = folium.Map(location=set_coordinates, zoom_start=25, width=800, height=500)

folium.Marker(set_coordinates, tooltip='You are here', icon=folium.Icon(color='red')).add_to(mymap)

for index, row in closest.iterrows():
    coord = row['Coordinates']
    folium.Marker(coord, tooltip=row['Name']).add_to(mymap)
    
closest = closest.drop(columns=['Coordinates'])

display(HTML(closest.to_html()))
display(mymap)

## This block makes a Map of all the (whatever your looking for)

In [ ]:
import folium

melbourne_coordinates = [-37.814, 144.96332]

m = folium.Map(location=melbourne_coordinates, zoom_start=16)

for _, row in df1.iterrows():
    lat, lng = row['Coordinates'][0], row['Coordinates'][1]
    folium.Marker(
        location=[lat, lng],
        tooltip=row['Name']
    ).add_to(m)

m

## This Block takes a user's address and shows any specified amount of the closest (whatever your looking for) to each other in a line and shows a map with a line.

In [ ]:
import folium
from scipy.spatial import distance_matrix

coords = df1['Coordinates'].tolist()
dist_matrix = distance_matrix(coords, coords)

def best_route_based_on_proximity(start_coordinates, n):
    starting_distances = [((x[0] - start_coordinates[0]) ** 2 + (x[1] - start_coordinates[1]) ** 2) ** 0.5 for x in coords]
    start_index = starting_distances.index(min(starting_distances))
    
    visited = set()
    path = [start_index]
    visited.add(start_index)
    
    current_index = start_index
    for _ in range(n - 1):
        min_distance = float('inf')
        next_index = -1
        for j, dist in enumerate(dist_matrix[current_index]):
            if j not in visited and dist < min_distance:
                min_distance = dist
                next_index = j
        if next_index == -1:
            break
        path.append(next_index)
        visited.add(next_index)
        current_index = next_index
    return path

address = input("Please enter the address: ")

api_key = "" # add your google api here


def get_coordinates_from_address(address, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            lat = data['results'][0]['geometry']['location']['lat']
            lng = data['results'][0]['geometry']['location']['lng']
            return lat, lng
    return None

start_coordinates = get_coordinates_from_address(address, api_key)

n = 10 # change this for a diffrent amount of (whatever your looking for)

route = best_route_based_on_proximity(start_coordinates, n)

m = folium.Map(location=start_coordinates, zoom_start=15)

folium.Marker(start_coordinates, tooltip="Starting Point", icon=folium.Icon(color='red')).add_to(m)

for i in route:
    lat, lng = coords[i]
    folium.Marker([lat, lng], tooltip=df1.iloc[i]['Name']).add_to(m)

for i in range(1, len(route)):
    folium.PolyLine([coords[route[i-1]], coords[route[i]]], color="blue", weight=2.5, opacity=1).add_to(m)

m